<a href="https://colab.research.google.com/github/jceresearch/paat/blob/master/charts_cheatsheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!Python3
import numpy as np
from scipy.stats import norm
import math
import pandas as pd
import random
import math
import numpy as np
from scipy.stats import loguniform
import datetime
from scipy.stats import poisson
from datetime import datetime, timedelta
print(datetime.now())
import pathlib
pathlib.Path('./test_data').mkdir(parents=True, exist_ok=True)
pathlib.Path('./output').mkdir(parents=True, exist_ok=True)


2021-06-26 10:09:25.982229


In [2]:
rows=[]
m=100000
rvs = loguniform.rvs(1e-3, 1, size=m)
p=poisson.rvs(5, size=m)
start_date= datetime.now() - timedelta(hours=24*180)
present=datetime.now()
create_date=start_date
p_tally=p[0]
p_pointer=0
for n in range(m):  
    if p_tally>0: 
        sec_offset=60*(60/p[p_pointer])
    else:
        p_pointer=p_pointer+1
        p_tally=p[p_pointer]
        if p[p_pointer]==0:
            sec_offset=60*60
        else:
            sec_offset=60*(60/p[p_pointer])
    p_tally=p_tally-1
    create_date=create_date + timedelta(seconds=sec_offset)
    if create_date > present:
        print("reached the present")
        break
    inc_type=random.choices(["incident","inc req","req",'alert'],[.4,.2,.2,.1],k=1)[0]
    prio= random.choices([1,2,3,4,5],[.001,.099,.2,.3,.4],k=1)[0]
    is_child=True if random.random() <.001 else False
    resolution_hs=rvs[n]*(prio**2)*24*10
    resolve_date=create_date + timedelta(hours=resolution_hs)
    if resolve_date > datetime.now():
            resolve_date=np.nan
            time_difference = datetime.now()-create_date
            #resolution_hs = time_difference / timedelta(hours=1)
            status= "open"
    else:
            status="closed"
    resolution_days=resolution_hs/24
    is_in_report=False if inc_type=="inc req" else True
    is_long_runner=True if resolution_days>=30 else False
    if is_long_runner:
        is_slow_runner=True
    else:
        is_slow_runner = True if resolution_days >10 else False     
    if status=='closed':
        outcome= random.choices(["fully",'failed','partial'],[.6,.02,.38],k=1)[0]
    else:
        outcome="open"
    resolution_bucket='a. <5 days'
    if resolution_days> 5:
        resolution_bucket='b. 6-20 days'
    if resolution_days> 20:
        resolution_bucket='c. 21-60 days'
    if resolution_days> 60:
        resolution_bucket='d. More 3 months'
    if resolution_hs> prio*8:
        made_sla=False
    else:
        made_sla=True 
        
    rows.append(["INC"+'{:06d}'.format(n+1), prio, create_date, resolve_date,resolution_days,resolution_hs,inc_type,is_in_report,is_child, is_long_runner,is_slow_runner,resolution_bucket, outcome, made_sla])

df=pd.DataFrame(rows, columns=["id",'prio',"date_created", 'date_resolved',"resolution_days",'resolution_hs','inc_type','is_in_report','is_child','is_long_runner','is_low_runner','resolution_bucket',"outcome","made_sla"])
df.tail()

reached the present


,id,prio,date_created,date_resolved,resolution_days,resolution_hs,inc_type,is_in_report,is_child,is_long_runner,is_low_runner,resolution_bucket,outcome,made_sla
21535,INC021536,5,2021-06-26 09:35:09.870965,NaT,7.295542,175.093010,incident,True,False,False,False,b. 6-20 days,open,False
21536,INC021537,4,2021-06-26 09:43:44.156679,NaT,24.385100,585.242397,inc req,False,False,False,True,c. 21-60 days,open,False
21537,INC021538,5,2021-06-26 09:52:18.442393,NaT,1.362901,32.709633,incident,True,False,False,False,a. <5 days,open,True
21538,INC021539,4,2021-06-26 10:00:52.728107,NaT,0.568954,13.654891,incident,True,False,False,False,a. <5 days,open,True
21539,INC021540,3,2021-06-26 10:09:27.013821,NaT,18.819402,451.665659,inc req,False,False,False,True,b. 6-20 days,open,False


In [3]:
df['yyyymm']=df.apply(lambda r: str(r['date_created'].year)+'{:02d}'.format(r['date_created'].month) ,axis=1)
dfg1=df[['yyyymm','id']].groupby('yyyymm').agg(len)
dfg1.reset_index(inplace=True)
dfg2= df.groupby('yyyymm')['made_sla'].apply(lambda x: round(np.sum(x)/len(x),2))
dfg2= dfg2.reset_index()


In [4]:
dfg1

,yyyymm,id
0,202012,438
1,202101,3724
2,202102,3416
3,202103,3673
4,202104,3571
5,202105,3629
6,202106,3089


In [5]:
dfg2

,yyyymm,made_sla
0,202012,0.29
1,202101,0.31
2,202102,0.31
3,202103,0.32
4,202104,0.31
5,202105,0.31
6,202106,0.31


In [7]:
df.to_excel('./test_data/data.xlsx')